In [ ]:
# leitura de dados
import pandas as pd

# PLN e padronização
from sklearn.feature_extraction.text import TfidfVectorizer

# modelos de classificação
from sklearn.ensemble import RandomForestClassifier

#pickles
import pickle


# consulta respostas banco (parte da automação realizada posteriormente)
#import FN_CONSULTA_BANCO as cb

tfidf = TfidfVectorizer(
        #este atributo coloca todas as palavras em minusculo
        lowercase=True,
        #este atributo exclui as palavras que aparecem menos que 5 vezes
        min_df = 5,
        max_features = 10000,
        #este atributo remove as stopwords **não utilizado**
        #stop_words=stopwords.words('portuguese')
    )

# RandomForestClassifier
# Instancia o Classificador
rfc = RandomForestClassifier()

In [ ]:
#----------------------------------------------------------------------------------------#
#Função: leitura_base_treino
#Descrição: Realiza leitura da base de treino que foi clusterizada por "MiniBatchKmeans",\ 
#          \Realiza Tratativa simples antes de treinar o modelo RandomForestClassifier
#          \Exporta o transformador "TfidfVectorizer" e o modelo RandomForestClassifier treinado arquivo pickle
#-----------------------------------------------------------------------------------------#
def leitura_base_treino():
    print("Inicio")
    
    # leitura da base de treino clusterizada por "MiniBatchKmeans" com 250.000 linhas
    dd = pd.read_excel("LOCAL_DO_ARQUIVO\TREINO_ATT.xlsx")

    # tratamento da base de treino
    dd['Resposta'] = [str(x) for x in dd['Resposta']]  # trasforma as respostas para o formato string
    dd['Resposta'] = [x.lower() for x in dd['Resposta']] # transforma todas as letras para minusculo
    dd = dd.dropna() # retira as respostas vazias
    dd=dd.reset_index() # reseta o indice do dataframe e cria uma coluna com os antigos índices
    dd = dd.drop(columns='index') # exclui a coluna dos índices

    # cria uma lista das respostas da base de treino
    X = list(dd['Resposta'])
    # cria uma lista das classes da base de treino
    y = list(dd['Clusters'])

    # treina e aplica o vetorizador na lista de respostas de treino
    X = tfidf.fit_transform(dd['Resposta'])
    
    #Criação do pickle ao qual armazena o TfidfVectorizer
    pickle.dump(tfidf, open("LOCAL_DO_ARQUIVO\tfidf.pkl", 'wb'))

    # treina o classificador
    rfc_new = rfc.fit(X, y)
                            
    #Criação do pickle ao qual armazena o Treino do modelo sobre aplicação de Random Florest Classifier                   
    pickle.dump(rfc_new, open("LOCAL_DO_ARQUIVO\rfc.pkl", 'wb'))

    print("Fim")
    
#Esta função será rodada apenas uma vez, visto que o treino do modelo é salvo\
#\em arquivo "pickle", podendo ser aberto posteriormente

In [ ]:
#----------------------------------------------------------------------------------------#
#Função: leitura_base_teste
#Descrição: Realiza leitura da base de teste que foi formatada no arquivo anterior,\ 
#          \Realiza Tratativa 
#          \Aplica transformação de palavras em vetores por "TfidfVectorizer"
#          \Aplica teste do modelo RandomForestClassifier
#          \Exporta base provinda do web_scrarespostas e clusters
#-----------------------------------------------------------------------------------------#
def leitura_base_teste():
    # leitura da base de teste
    print("Inicio 1")

    #Leitura do excel provindo do arquivo "Recebe_email"
    #Ressalvo que o processo de comunicação entre os arquivos (para automatização) foi feita tardiamente
    dt = pd.read_csv("LOCAL_DO_ARQUIVO\RESPOSTAS_EMAIL.csv", sep = None, engine = "python")

    #tratamento da base de teste
    dt['Resposta'] = [str(x) for x in dt['Resposta']]  #Trasforma as respostas para o formato string
    dt['Resposta'] = [x.lower() for x in dt['Resposta']]  #Transforma todas as letras para minusculo
    dt = dt.dropna()  #Retira as respostas vazias
    dt = dt.reset_index()  #Reseta o indice do dataframe e cria uma coluna com os antigos índices
    dt = dt.drop(columns='index')  #Exclui a coluna dos índices

    #Tratamento expecífico para erros que a base de e-mails entregue pelo cliente X possuia
    #Mesmo tratamento realizado no arquivo anterior
    rr = open("LOCAL_DO_ARQUIVO\certo.txt", "r", encoding='utf-8') 
    ww = open("LOCAL_DO_ARQUIVO\errado.txt", "r", encoding='utf-8')
    aux = rr.readlines()
    certo = []
    for i in range(len(aux)):
        certo.append(aux[i].rstrip("\n'"))
    aux2 = ww.readlines()
    errado = []
    for i in range(len(aux2)):
        errado.append(aux2[i].rstrip("\n'"))
    for i in range(len(certo)):
        dt['Resposta'].replace(regex=True, inplace=True, to_replace=errado[i], value=certo[i])

    #Retirada de palavras de recepção, como "bom dia", "boa tarde", "olá" entre outros
    delet = open("LOCAL_DO_ARQUIVO\words.txt", "r", encoding='utf-8')
    for i in delet:
        dt['Resposta'].replace(regex=True, inplace=True, to_replace=i, value = " ")
    dt.dropna()
    dt = dt.reset_index()
    dt = dt.drop(columns='index')

    #cria uma lista das respostas da base de teste
    q = list(dt['RESPOSTA'])
    
    # Importa vetorização "TfidfVectorizer" da função anterior 
    vec = pickle.load(open("LOCAL_DO_ARQUIVO\tfidf.pkl", 'rb'))

    # Aplica o vetorizador na lista de respostas de teste
    q = vec.transform(q)

    # Importa Random Florest Classifier da função anterior (modelo treinado)
    rfc = pickle.load(open("LOCAL_DO_ARQUIVO\rfc.pkl", 'rb'))

    # Aplica o classificador nos dados de teste, a variavel "resultado_rfc" cria um array com as classes de cada resposta
    resultado_rfc = rfc.predict(q)
    
    # Modifica o array de classes para o tipo lista
    result = resultado_rfc
    result = list(result)
    
    # Atribui essa lista de classes para uma nova coluna do dataframe de teste
    dt['CLUSTERS'] = result
    
    # Exporta o dataframe de teste ja com a coluna nova
    dt.to_excel("LOCAL_DO_ARQUIVO\FINALRFC.xlsx")
    print("Fim 2")
